En este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

In [1]:
import pandas as pd
import numpy as np
import sidetable
import re

In [2]:
df=pd.read_csv('../EDA/datos/attack_limpio.csv', index_col=0)

In [3]:
df.head(2)

,case_number,year,country,age,time,species_,mes,fatal_,sex2
0,1800.00.00,1800,seychelles,NaN,NaN,NaN,NaN,y,F
1,1797.05.28.R,1797,NaN,NaN,NaN,NaN,May,y,NaN


1. Columna de species: 
   
   Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.
   - Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías
    que hemos definido previamente.
         
        *imaginemos que el valor de una celda es el siguiente

                    'White shark, 3.5 m'

        *tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
        Un patrón que podriamos usar es: 

                    patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

        *de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

    - Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [4]:
patronblanco= '.*([W|w]hite|HITE).*'

patrontigre= '.*([T|t]iger|IGER).*'

patronbull= '.*([B|b]ull|ULL).*'

patrongrey= '.*([G|g]rey|REY).*'

patronlemon= '.*([L|l]emon|LEMON).*'


In [7]:
df=df.fillna("NAN")

In [8]:
#Pair Lucia Lara Anais 
def tiburon(val):

    if re.findall(patronblanco,val):
        df['cleaned_species']= 'White'
        return val.replace(val,'White')
    elif re.findall(patrontigre,val):
        df['cleaned_species']= 'Tiger'
        return val.replace(val,'Tiger')  
    elif re.findall(patronbull,val):
        df['cleaned_species']= 'Bull'
        return val.replace(val,'Bull')
    elif re.findall(patrongrey,val):
        df['cleaned_species']= 'Grey'
        return val.replace(val,'Grey')
    elif re.findall(patronlemon,val):
        df['cleaned_species']= 'Lemon'
        return val.replace(val,'Lemon')
    else:
         return val.replace(val,'Undefined') 
    

In [9]:
df['cleaned_species']=df['species_'].apply(tiburon)

In [16]:
df.sample(10)

,case_number,year,country,age,time,species_,mes,fatal_,sex2,cleaned_species
301,2017.06.05,2017,french polynesia,NAN,NAN,NAN,NAN,n,M,Undefined
6278,1805.09.00,1805,usa,NAN,NAN,Shark involvement prior to death unconfirmed,Sep,NAN,M,Undefined
2368,1998.01.17,1998,australia,NAN,Afternoon,NAN,Jan,n,M,Undefined
2238,2000.02.01,2000,australia,26,NAN,3 m [10'] shark,Feb,n,M,Undefined
5126,1933.04.10,1933,usa,24,NAN,Possibly a bull shark or tiger shark,Apr,y,M,Tiger
396,2016.08.29.b,2016,usa,25,15h00,"Bull shark, 6'",Aug,n,M,Bull
3959,1962.08.19,1962,usa,40,15h30,Unknown,Aug,y,M,Undefined
5695,1901.12.01,1901,australia,10,NAN,NAN,Dec,n,M,Undefined
3159,1982.00.00.a,1982,tunisia,NAN,NAN,NAN,NAN,NAN,NAN,Undefined
4719,1947.12.00,1947,senegal,NAN,11h00,Tiger shark,Dec,y,M,Tiger


2. Columna de age: 
   
   es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
   - Edad en formato string
   - Edades separadas por &, or, to, >
   - Edades con ?
   Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
   - Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
   - Puede que os salga un error similar a este:
  
                     TypeError: expected string or bytes-like object
                  
     Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

                     df['nombre_columna'] = df['nombre_columna'].astype(str)
                     
   - Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. 
   - Por último cambiad el tipo de la columna de string a integer.
  


In [42]:
df['age']=df['age'].astype(str)


In [43]:
edades=list(df['age'].unique())
print(edades)


['NAN', 'young', '14', '19', '20', '15', '21', '27', '36', '25', '6', '16', '50', '13 or 14', '57', '11', '48', '18', '52', '12', '32', '10', '34', '30', '60', '33', '29', '54', '41', '37', '56', '69', '38', '55', '35', '46', '45', '40s', '28', '24', '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8', '82', '73', '20s', '68', '51', '39', '58', 'Teen', '47', '61', '65', '66', '43', '60s', '9', '72', '59', '64', '23', '71', '44', '62', '63', '70', '18 months', '53', '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20', '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26', '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75', '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28', '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35', '23 & 26', '1', '(adult)', '33 & 37', '25 or 28', '37, 67, 35, 27,  ? & 27',

In [141]:
def edad(x):
    if re.findall('\d+',x):
        if re.findall('[^s]',x):
            if re.findall('\d+[^\&|(or)|\>|\<|(to)]',x):

                    return x

In [142]:
df['new']=df['age'].apply(edad)

In [143]:
print(df['new'].unique())

[None '14' '19' '20' '15' '21' '27' '36' '25' '16' '50' '13 or 14' '57'
 '11' '48' '18' '52' '12' '32' '10' '34' '30' '60' '33' '29' '54' '41'
 '37' '56' '69' '38' '55' '35' '46' '45' '40s' '28' '24' '26' '49' '22'
 '31' '17' '40' '13' '42' '82' '73' '20s' '68' '51' '39' '58' '47' '61'
 '65' '66' '43' '60s' '72' '59' '64' '23' '71' '44' '62' '63' '70'
 '18 months' '53' '30s' '50s' '77' '74' '28 & 26' '86' '18 or 20'
 '12 or 13' '46 & 34' '28, 23 & 30' '36 & 26' '8 or 10' '84' '30 or 36'
 '6½' '21 & ?' '75' '33 or 37' 'mid-30s' '23 & 20' ' 30' '7      &    31'
 ' 28' '20?' "60's" '32 & 30' '16 to 18' '87' '67' 'mid-20s' 'Ca. 33'
 '74 ' '45 ' '21 or 26' '20 ' '>50' '18 to 22' '9 & 12' '? & 19'
 '9 months' '25 to 35' '23 & 26' '33 & 37' '25 or 28'
 '37, 67, 35, 27,  ? & 27' '21, 34,24 & 35' '30 & 32' '50 & 30' '17 & 35'
 '13 or 18' '34 & 19' '33 & 26' '2 to 3 months' ' 43' '81' '7 or 8' '78'
 '17 & 16' 'Both 11' '9 or 10' '36 & 23' '?    &   14' '10 or 12'
 '31 or 33' '2½']


In [123]:
df.sample(20)

,case_number,year,country,age,time,species_,mes,fatal_,sex2,cleaned_species,new
1692,2005.11.16.R,2005,new caledonia,NAN,NAN,4.5 m white shark,Nov,n,NAN,White,None
4959,1939.01.12,1939,australia,Both 11,NAN,7' shark,Jan,n,M,Undefined,Both 11
69,ND-0122,0,usa,NAN,NAN,NAN,NAN,n,M,Undefined,None
3682,1967.07.05,1967,turkey,36,13h40,NAN,Jul,y,M,Undefined,36
4343,1957.12.20,1957,south africa,15,16h00,NAN,Dec,y,M,Undefined,15
1271,2009.07.24.b,2009,usa,14,Afternoon,NAN,Jul,n,F,Undefined,14
2998,1985.11.03,1985,usa,24,NAN,NAN,Nov,n,M,Undefined,24
2529,1995.07.28.c,1995,usa,18,14h10,NAN,Jul,n,M,Undefined,18
1766,2005.03.25,2005,venezuela,NAN,17h30,NAN,Mar,n,M,Undefined,None
1780,2005.01.08,2005,new zealand,39,12h00,White shark,Jan,n,M,White,39


3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.